# Mutabakat

 ## Veri Manipülasyonu Tamamlanan aşamalar
* Veri setleri csv versiyonuyla veri çerçevesi(DataFrame, df) olarak yüklendi
    *  Sedna, Booking, Expedia
* Sedna 
    * "T.Oda Fiyatı" sütunu ',' yerine '.' kullanıyor
* Booking
    * 'on_bad_lines' parametresi 'skip' olarak atandı.
    * 'Price' sütunundan 'EUR' kaldırıldı.
    * 'Price' ve 'Book number' sütunları 'T.Oda Fiyatı' ve 'Voucher' olarak yeniden isimlendirildi.
* Expedia
    * 'on_bad_lines' parametresi 'skip' olarak atandı.
    * 'Price' sütunundan 'EUR' kaldırıldı.
    * 'Rezervasyon Tutarı' ve 'Reservation no.' sütunları 'T.Oda Fiyatı' ve 'Voucher' olarak yeniden isimlendirildi. 

In [ ]:
import pandas as pd

## VERİ OKUMA

### Sedna Datasını Okuma

İlk satır bilgi olarak sadece "Reservation Info" stringi tuttuğu, son satır da bilgi olarak sadece "Sum =(Toplam Değer)" tuttuğu için kaldırılmıştır.

In [ ]:
sedna_df = pd.read_csv("./SednaUtf8QuoteOff.csv",on_bad_lines='skip',sep=';', usecols=['Voucher','T.Oda Fiyatı'],skipfooter=1, engine='python')
booking_df = pd.read_excel("./Booking.xls", usecols=['Book number', 'Price', 'Status', 'Commission amount'])
expedia_df = pd.read_csv("./reservationsList.csv", usecols=['Rezervasyon no.','Rezervasyon tutarı'])

In [ ]:
sedna_df.head()

In [ ]:
booking_df.head()

In [ ]:
expedia_df.head()

Integer geçişlerinde desimal değerlerde problem yaşamamamk için . ve virgülün yerleri değiştirilmiştir. Örn : "1.440,0" olan değer önce "1440,0", ardından "1440.0" olarak çevirilerek veri kaybı ve tutarsızlığı dengelenmiştir.

In [ ]:
sedna_df['T.Oda Fiyatı'] = sedna_df['T.Oda Fiyatı'].str.replace('.','')
sedna_df.head()

In [ ]:
sedna_df['T.Oda Fiyatı'] = sedna_df['T.Oda Fiyatı'].str.replace(',','.')
sedna_df.head()

"Voucher" ve "T.Oda Fiyatı" sütunlarının veri tipleri düzenlemesi

In [ ]:
sedna_df["Voucher"] = sedna_df["Voucher"].astype(str)
sedna_df["T.Oda Fiyatı"] = sedna_df["T.Oda Fiyatı"].astype(float)
sedna_df= sedna_df.groupby('Voucher')['T.Oda Fiyatı'].sum().reset_index()

In [ ]:
sedna_df

# Booking Datasını Okuma

Indeksler, sedna_df'de denk gelen değerlere sahip indekslerle isim olarak eşlendi. Bu rapor için gerekli sütunlar yüklendi. Komisyon ölçümlerinde farklılıkları önlemek için ayrı dataframe oluşturuldu.

In [ ]:
booking_df = booking_df.rename(columns={'Book number': 'Voucher','Price' : 'T.Oda Fiyatı'})
commisions_booking_df = booking_df.copy()
booking_df.head()

##### String Değerlerini Kaldırma

String değer tutan "T.Oda Fiyatı" ve "Commission amount" sütunlarından " EUR" değerleri, float veri ile matematiksel işlem imkanları için kaldırıldı.

In [ ]:
commisions_booking_df['T.Oda Fiyatı'] = commisions_booking_df['T.Oda Fiyatı'].str.replace(' EUR','')
commisions_booking_df['Commission amount'] = commisions_booking_df['Commission amount'].str.replace(' EUR','')

### Veri Tipi Düzenleme

"Voucher", "T.Oda Fiyatı", "Commission amount" sütunnlarının tipleri veri kaybı ve tutarsızlığı önlemek için değiştirildi.

In [ ]:
commisions_booking_df["Voucher"] = commisions_booking_df["Voucher"].astype(str)
commisions_booking_df["T.Oda Fiyatı"] = commisions_booking_df["T.Oda Fiyatı"].astype(float)
commisions_booking_df["Commission amount"] = commisions_booking_df["Commission amount"].astype(float)

In [ ]:
booking_df = commisions_booking_df[(commisions_booking_df['Status'] == 'ok')]

In [ ]:
booking_df = commisions_booking_df[['Voucher','T.Oda Fiyatı']]

expedia_df = pd.read_excel("./ExpediaDec.xlsx",usecols=['Rezervasyon no.','Rezervasyon tutarı','Durum'])
expedia_df= expedia_df.rename(columns={'Rezervasyon no.':'Voucher','Rezervasyon Tutari': 'T.Oda Fiyatı','Durum': 'Status'})
expedia_df["Voucher"] = expedia_df["Voucher"].astype(str)
expedia_df["T.Oda Fiyatı"] = expedia_df["T.Oda Fiyatı"].astype(float)

### Veri Tipi Değiştirme

In [ ]:
booking_df.loc[:, "T.Oda Fiyatı"] = booking_df["T.Oda Fiyatı"].astype(float).round(2)

In [ ]:
komisyon_df = commisions_booking_df[(commisions_booking_df['Status'] != 'ok') & (~commisions_booking_df['Commission amount'].isna())]
komisyon_df.reset_index(drop=True, inplace=True)

# Farkları Çıkartma

"Voucher", "T.Oda Fiyatı", "Commission amount" sütunnlarının tipleri veri kaybı ve tutarsızlığı önlemek için değiştirildi.

In [ ]:
booking_df.head()

In [ ]:
bookingdeVar_sednadaYok = booking_df[(commisions_booking_df['Status'] == 'ok') & (~booking_df['Voucher'].isin(sedna_df['Voucher']))]
bookingdeVar_sednadaYok

In [ ]:
booking_fark = sedna_df.merge(booking_df, on='Voucher')
booking_fark= booking_fark.rename(columns={'T.Oda Fiyatı_x':'Sedna Tutarı','T.Oda Fiyatı_y': 'Booking.com Tutarı'})

Sedna - Booking 0 dan farklı olanları yazdır

In [ ]:
booking_fark['Booking.com Tutarı'] = (booking_fark['Booking.com Tutarı']).round(2)
booking_fark['FARK'] = booking_fark['Sedna Tutarı'] - booking_fark['Booking.com Tutarı']
booking_fark = booking_fark[booking_fark['FARK'] != 0]

### Vergi Ekleme


Booking Datasına %2 vergiyi ekleyip 0'dan farklı olanı yazdır

In [ ]:
#################    KESİLEN %2 VERGİ   ###########
vergi = 1.01818

In [ ]:
booking_fark['Booking.com Tutarı Vergili'] = (booking_fark['Booking.com Tutarı'] * vergi).round(2)
booking_fark['FARK Vergili'] = booking_fark['Sedna Tutarı'] - booking_fark['Booking.com Tutarı Vergili']
booking_fark = booking_fark[booking_fark['FARK Vergili'] != 0]

# Expedia Datasını Okuma

In [ ]:
expedia_df = expedia_df.rename(columns={'Rezervasyon no.': 'Voucher','Rezervasyon tutarı' : 'T.Oda Fiyatı'})
expedia_df.head()

In [ ]:
expedia_df["Voucher"] = expedia_df["Voucher"].astype(str)
expedia_df["T.Oda Fiyatı"] = expedia_df["T.Oda Fiyatı"].astype(float)

In [ ]:
expedia_fark = sedna_df.merge(expedia_df, on='Voucher')
expedia_fark= expedia_fark.rename(columns={'T.Oda Fiyatı_x':'Sedna Tutarı','T.Oda Fiyatı_y': 'Expedia Tutarı'})
expedia_fark.head()

In [ ]:
expedia_fark['FARK'] = expedia_fark['Sedna Tutarı'] - expedia_fark['Expedia Tutarı']
expedia_fark = expedia_fark[expedia_fark['FARK'] != 0]
expedia_fark

In [ ]:
expediadaVar_sednadaYok = expedia_df[~expedia_df['Voucher'].isin(sedna_df['Voucher'])]
expediadaVar_sednadaYok = expediadaVar_sednadaYok[expediadaVar_sednadaYok['T.Oda Fiyatı'] != 0] # 0 olanlar cancel
expediadaVar_sednadaYok

## BOOKING.COM FARKLILIKLARI

In [ ]:
booking_fark

### Cancel

In [ ]:
komisyon_df

### Sednada Yok


In [ ]:
bookingdeVar_sednadaYok

## EXPEDIA FARKLILIKLARI

In [ ]:
expedia_fark

In [ ]:
expediadaVar_sednadaYok